# Real Options Analysis (ROA) Example

This notebook demonstrates how to perform real options analysis to value flexibility in health technology decisions under uncertainty.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Import the ROA model
from roa_engine import RealOptionsEngine

# Initialize the engine with default parameters
roa_engine = RealOptionsEngine(risk_free_rate=0.03, time_horizon=5.0)

In [ ]:
# Define parameters for ROA analysis
parameters = {
    'strategies': ['ECT', 'IV-KA', 'PO-KA'],
    'perspective': 'health_system',
    'option_types': ['delay', 'abandon', 'expand'],
    'wtp_threshold': 50000,
    'time_horizon': 5,
    'risk_free_rate': 0.03
}

In [ ]:
# Define mock PSA data for ROA analysis
# In practice, this would come from PSA results
np.random.seed(42)
strategies = ['ECT', 'IV-KA', 'PO-KA']
n_draws = 1000

mock_psa_data = []
for strategy in strategies:
    for i in range(n_draws):
        # Generate mock cost and effect data
        cost = np.random.normal(1000 if strategy == 'ECT' else 1500 if strategy == 'IV-KA' else 1200, 100)
        effect = np.random.normal(0.5 if strategy == 'ECT' else 0.7 if strategy == 'IV-KA' else 0.6, 0.05)
        
        mock_psa_data.append({
            'draw': i,
            'strategy': strategy,
            'cost': max(cost, 100),  # Ensure positive costs
            'effect': max(effect, 0.1)  # Ensure positive effects
        })

psa_df = pd.DataFrame(mock_psa_data)
psa_df['perspective'] = 'health_system'

In [ ]:
# Run the ROA analysis
results = roa_engine.analyze(
    psa_df,
    strategies=parameters['strategies'],
    perspective=parameters['perspective'],
    option_types=parameters['option_types'],
    wtp_threshold=parameters['wtp_threshold']
)

In [ ]:
# Display results
print("Base Case NPVs:")
for strategy, npv in results.risk_adjusted_npv.items():
    print(f"  {strategy}: ${npv:,.2f}")

print("\nValue of Flexibility:")
for strategy, value in results.value_of_flexibility.items():
    print(f"  {strategy}: ${value:,.2f}")

print("\nTotal Value with Options:")
for strategy, value in results.total_value_with_options.items():
    print(f"  {strategy}: ${value:,.2f}")

In [ ]:
# Visualize results
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Base vs Total Value
strategies = list(results.risk_adjusted_npv.keys())
base_values = [results.risk_adjusted_npv[s] for s in strategies]
total_values = [results.total_value_with_options[s] for s in strategies]
flex_values = [results.value_of_flexibility[s] for s in strategies]

x = range(len(strategies))
ax[0].bar([i - 0.2 for i in x], base_values, width=0.4, label='Base NPV', alpha=0.7)
ax[0].bar([i + 0.2 for i in x], total_values, width=0.4, label='Total Value with Options', alpha=0.7)
ax[0].set_xlabel('Strategy')
ax[0].set_ylabel('Value ($AUD)')
ax[0].set_title('Base NPV vs Total Value with Real Options')
ax[0].set_xticks(x)
ax[0].set_xticklabels(strategies, rotation=45)
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Plot 2: Value of Flexibility
ax[1].bar(x, flex_values, color='orange', alpha=0.7)
ax[1].set_xlabel('Strategy')
ax[1].set_ylabel('Value of Flexibility ($AUD)')
ax[1].set_title('Value of Managerial Flexibility')
ax[1].set_xticks(x)
ax[1].set_xticklabels(strategies, rotation=45)
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Next Steps

1. Configure the analysis with real PSA data
2. Run the analysis with appropriate option types
3. Interpret the value of flexibility for each strategy
4. Consider different time horizons and risk-free rates